In [ ]:
"""
Author: Daniel Wu
Last Updated: September 22th, 2020
Purpose: Execute The Bansal and Yaron Long Run Risk model:
         Set parameters
         Find Solutions
         Run Simulations         
"""

import numpy as np
import math
from numpy import linalg 
import pandas as pd
from operator import itemgetter
import importlib
import import_ipynb
from matplotlib import pyplot as plt

def pause():
    programPause = input("Press the <ENTER> key to continue or type any letter to break loop")
    return programPause

np.random.seed(60201)  

In [ ]:
####################################
# A2 - Function to find solutions ##
####################################

# keyword arguments (only need to pass dictionary to the function)

def Solution(**p):  
    
    ## CONSUMPTION CLAIM

    kappa_1 = np.random.uniform(0,1)

    z_bar = np.log(kappa_1/(1-kappa_1))
    kappa_0 = np.log(1+np.exp(z_bar)) - kappa_1*z_bar

    tol = 1000
    iter_c = 0
    
    while tol > 1e-15 and iter_c <= 100000:
        
        iter_c = iter_c + 1    
        
        A2 = -(((p['gamma']-1)*(1-1/p['psi']))/(2*(1-kappa_1*p['nu']))) * \
                (1+((kappa_1*p['phi_e'])/(1-kappa_1*p['rho']))**2)
        
        A0 = 1/(1 - kappa_1)*(np.log(p['delta']) + \
             kappa_0 + \
             (1-1/p['psi'])*p['mu_c'] + \
             kappa_1*A2*(1-p['nu'])*(p['sig_bar']**2) + \
             (p['theta']/2)*(kappa_1*A2*p['sig_w'])**2)
                                
        z_bar = A0 + A2*p['sig_bar']**2
        
        kappa_1_new = np.exp(z_bar)/(1+np.exp(z_bar))        
        tol = abs(kappa_1_new - kappa_1)  
        
        kappa_1 = kappa_1_new                            
        kappa_0 = np.log(1+np.exp(z_bar)) - kappa_1*z_bar 
    
    if iter_c <= 100000:
        print(f'Consumption Algo Converged at Iteration {iter_c}')
    else:
        print('Consumption Algo Not Converged')
    
    A1 = (1-1/p['psi'])/(1-kappa_1*p['rho'])
    
    # Market prices of risk
    
    lambda_eta = p['gamma']
    lambda_e = (1-p['theta'])*kappa_1*A1*p['phi_e']    
    lambda_w = (1-p['theta'])*kappa_1*A2    
        
    ## DIVIDEND CLAIM
    
    kappa_1_m = np.random.uniform(0,1)
    
    z_bar_m = np.log(kappa_1_m/(1-kappa_1_m))
    kappa_0_m = np.log(1+np.exp(z_bar_m)) - kappa_1_m*z_bar_m
    
    cap_gamma_2 = (p['theta']-1)*(kappa_1*p['nu']-1)*A2
    cap_gamma_0 = np.log(p['delta']) - (1/p['psi'])*p['mu_c'] - \
                  (p['theta']-1)*(A2*(1-p['nu'])*p['sig_bar']**2 + (p['theta']/2)*(kappa_1*A2*p['sig_w'])**2)
    
    tol_m = 1000
    iter_m = 0
    
    while tol_m > 1e-15 and iter_m <= 100000:
        
        iter_m = iter_m + 1    
        
        A1_m = (p['phi'] - 1/p['psi'])/(1-kappa_1_m*p['rho'])
        
        A2_m = (1/(1-kappa_1_m*p['nu']))*(cap_gamma_2 + \
                                     0.5*((p['pi']-lambda_eta)**2 + \
                                     (kappa_1_m*A1_m*p['phi_e']-lambda_e)**2))
        
        A0_m = 1/(1-kappa_1_m)*(cap_gamma_0 + \
                                kappa_0_m + \
                                p['mu_d'] + \
                                kappa_1_m*A2_m*(1-p['nu'])*p['sig_bar']**2 + \
                                0.5*((kappa_1_m*A2_m-lambda_w)**2)*(p['sig_w']**2))        
         
        z_bar_m = A0_m + A2_m*p['sig_bar']**2
        
        kappa_1_m_new = np.exp(z_bar_m)/(1+np.exp(z_bar_m))        
        tol_m = abs(kappa_1_m_new - kappa_1_m)
        
        kappa_1_m = kappa_1_m_new
        kappa_0_m = np.log(1 + np.exp(z_bar_m)) - kappa_1_m*z_bar_m                
    
    if iter_m <= 100000:
        print(f'Dividend Algo Converged at Iteration {iter_m}')
    else:
        print('Dividend Algo Not converged')
        
    beta_eta = p['pi']
    beta_e = kappa_1_m*A1_m*p['phi_e']
    beta_w = kappa_1_m*A2_m
    
    #used to calculate risk-free rate
 
    B = kappa_1*A1*p['phi_e']
    
    lambda_m_eta = -p['gamma']
    lambda_m_e = (1-p['theta'])*B
    lambda_m_w = (1-p['theta'])*A2*kappa_1    
    
    # Return solution as a Dictionary  
    
    return {'A0':A0, 
            'A1':A1, 
            'A2':A2, 
            'kappa_0':kappa_0, 
            'kappa_1':kappa_1, 
            'lambda_eta':lambda_eta, 
            'lambda_e':lambda_e,
            'lambda_w':lambda_w,
            'A0_m':A0_m,
            'A1_m':A1_m,
            'A2_m':A2_m,
            'kappa_0_m':kappa_0_m,
            'kappa_1_m':kappa_1_m,
            'beta_eta':beta_eta,
            'beta_e':beta_e,
            'beta_w':beta_w,
            'cap_gamma_0':cap_gamma_0,
            'cap_gamma_2':cap_gamma_2,
            'lambda_m_eta':lambda_m_eta,
            'lambda_m_e':lambda_m_e,
            'lambda_m_w':lambda_m_w,
            'B':B
           }

In [ ]:
################################
## A3 - Population Simulation ##
################################

# process length (months)
# Python is much slower than Matlab. Here's a rough guideline:
# Tpop = 10000 - 30 seconds to run pop simulation
# Tpop = 100000 - 2- 3 minutes to run pop simulation

def pop_sim(**p):
    
    theta = (1-p['gamma'])/(1-1/p['psi']) 
        
    # Shocks

    eta_pop = np.random.normal(0, 1, size=(1, p['Tpop']))
    e_pop = np.random.normal(0, 1, size=(1, p['Tpop']))
    w_pop = np.random.normal(0, 1, size=(1, p['Tpop']))
    ud_pop = np.random.normal(0, 1, size=(1, p['Tpop']))

    # Consumption and Div process (log monthly growth)

    x_pop = np.zeros(p['Tpop'])
    delc_pop = np.zeros(p['Tpop'])
    deld_pop = np.zeros(p['Tpop'])

    sig2_pop = np.zeros(p['Tpop'])
    sig2_pop[0] = p['sig_bar']**2 + p['sig_w']*w_pop[0][0]

    delc_pop[0] = p['mu_c'] + np.sqrt(sig2_pop[0])*eta_pop[0][0] # init cons. growth (no persistent term)
    deld_pop[0] = p['mu_d'] + p['pi']*np.sqrt(sig2_pop[0])*eta_pop[0][0] \
                            + p['xi']*np.sqrt(sig2_pop[0])*ud_pop[0][0] # init div. growth (no persistent term)

    # Monthly Growth Simulation

    # Note: The range function has the following syntax:
    # range(starting_index, end_index, incrementer)  
    # 1 is the second index of the process (since Python starts at 0)
    # Once it reaches the end_index (i.e. 1,200,000), the loop exits
    # so the last index is 1,199,999

    for i in range(1, p['Tpop'], 1):    

        sig2_pop[i] = p['sig_bar']**2 + p['nu']*(sig2_pop[i-1] - \
                                   p['sig_bar']**2) + \
                                   p['sig_w']*np.sqrt(sig2_pop[i-1])*w_pop[0][i] # sim cons vol (case 2)
                                                                            # using CIR to ensure
                                                                            # no negative vol

        x_pop[i] = p['rho']*x_pop[i-1] + p['phi_e']*np.sqrt(sig2_pop[i-1])*eta_pop[0][i]   # sim growth persistence    

        delc_pop[i] = p['mu_c'] + x_pop[i-1] + np.sqrt(sig2_pop[i-1])*eta_pop[0][i]   # sim cons growth

        deld_pop[i] = p['mu_d'] + p['phi']*x_pop[i-1] + \
                             p['pi']*np.sqrt(sig2_pop[i-1])*eta_pop[0][i] + \
                             p['xi']*np.sqrt(sig2_pop[i-1])*ud_pop[0][i] # sim div growth

    # Price-Cons ratio process

    z_pop = p['A0'] + p['A1']*x_pop + p['A2']*sig2_pop
    zm_pop = p['A0_m'] + p['A1_m']*x_pop + p['A2_m']*sig2_pop

    # Returns

    rc_pop = np.zeros(p['Tpop'])
    rc_pop[0] = None 

    rd_pop = np.zeros(p['Tpop'])
    rd_pop[0] = None 

    rf_pop = np.zeros(p['Tpop'])
    rf_pop[0] = None

    var_sdf = (((p['lambda_m_eta'])**2 + (p['lambda_m_e'])**2)*sig2_pop + ((p['lambda_m_w'])**2)*p['sig_w']**2)    

    E_rc = p['kappa_0'] + p['kappa_1']*(p['A0']+p['A1']*p['rho']*x_pop \
                  + p['A2']*(p['sig_bar']**2+p['nu']*(sig2_pop-p['sig_bar']**2))) \
                  + p['mu_c'] + x_pop - z_pop

    # Monthly Return Simulation (cons and div via Campbell-Shiller, riskfree via SDF)

    for i in range(1, p['Tpop'], 1):
        rc_pop[i] = p['kappa_0'] + p['kappa_1']*z_pop[i] + delc_pop[i] - z_pop[i-1]
        rd_pop[i] = p['kappa_0_m'] + p['kappa_1_m']*zm_pop[i] + deld_pop[i] - zm_pop[i-1]   

        # riskfree rate (from q1)
        rf_pop[i] = - p['theta']*np.log(p['delta']) \
                    + (p['theta']/p['psi'])*(p['mu_c'] + x_pop[i]) \
                    + (1-p['theta'])*(E_rc[i]) \
                    - 0.5*var_sdf[i]
    
    return {'eta_pop':eta_pop, 
            'e_pop':e_pop, 
            'w_pop':w_pop, 
            'ud_pop':ud_pop, 
            'x_pop':x_pop, 
            'delc_pop':delc_pop,
            'deld_pop':deld_pop,
            'sig2_pop':sig2_pop,
            'z_pop':z_pop,
            'zm_pop':zm_pop,
            'rc_pop':rc_pop,
            'rd_pop':rd_pop,
            'rf_pop':rf_pop
           }

In [ ]:
# ANNUALIZE MONTHLY PROCESSES

def agg_flow(monthly_strip, length):
    
    Tpop_ann = int(length/12) # Annual process
    
    annual_strip = np.zeros(int(len(monthly_strip)/12))    
    annual_strip[0] = None # initialize first period cons growth    
    
    for i in range(1, len(annual_strip), 1):

        den = 0
        nom = 0    
        den_m = 0
        nom_m = 0

        anchor = monthly_strip[12*i-12]
        
        for j in range(24):        

            if j < 12:
                den = den + np.exp(np.sum(monthly_strip[(12*i-12):(12*i-12)+j]) - np.sum(anchor))
        
            if j >= 12:
                nom = nom + np.exp(np.sum(monthly_strip[(12*i-12):(12*i-12)+j]) - np.sum(anchor))
        
        annual_strip[i] = np.log(nom/den)
        
    return annual_strip


def agg_flow_ret(monthly_strip, length):
    
    Tpop_ann = int(length/12) # Annual process
    
    annual_strip = np.zeros(Tpop_ann)
    annual_strip[0]= None

    for i in range(1, Tpop_ann, 1):
        annual_strip[i] = np.sum(monthly_strip[(12*i-12):(12*i-12)+11])
        
    return annual_strip
    

In [ ]:
#####################
## A4 - Simulation ##
#####################

# Computational cost guideline:
# Nsim = 10000 : 10 minutes to run A4 simulation
# Nsim = 1000 :  2 minutes to run 
# Nsim = 100 :  <30 seconds to run


def mc_sim(**p):

    # shocks
    eta_sim = np.random.normal(0, 1, size = (p['Tsim'], p['Nsim']))
    e_sim = np.random.normal(0, 1, size = (p['Tsim'], p['Nsim']))
    w_sim = np.random.normal(0, 1, size = (p['Tsim'], p['Nsim']))
    ud_sim = np.random.normal(0, 1, size = (p['Tsim'], p['Nsim']))                         

    # consumption volatility (case 2)

    sig2_sim = np.zeros((p['Tsim'], p['Nsim']))
    sig2_sim[0,:] = p['sig_bar']**2 + p['sig_w']*w_sim[0,:]

    for i in range(1, p['Tsim'], 1):
        sig2_sim[i,:] = p['sig_bar']**2 + p['nu']*(sig2_sim[i-1,:] \
                                        - p['sig_bar']**2) \
                                        + p['sig_w']*np.sqrt(sig2_sim[i-1,:])*w_sim[i,:]

    # Consumption and Div growth

    x_var = (p['phi_e']**2)*(sig2_sim)/(1-p['rho']**2)
    x_sim = np.random.normal(0, 1, size = (p['Tsim'], p['Nsim'])) * np.sqrt(x_var)

    delc_sim = np.zeros((p['Tsim'], p['Nsim']))
    delc_sim[0,:] = p['mu_c'] + np.sqrt(sig2_sim[0,:])*eta_sim[0,:]

    deld_sim = np.zeros((p['Tsim'], p['Nsim']))
    deld_sim[0,:] = p['mu_d'] + p['pi']*np.sqrt(sig2_sim[0,:])*eta_sim[0,:] \
                              + p['xi']*np.sqrt(sig2_sim[0,:])*ud_sim[0,:]
    
    # This setup (from A4_simulation.m) generates every walk simultaneously
    # like parallel universes rather than process by process. Very cool!    
    
    for i in range(1, p['Tsim'], 1):    
        x_sim[i,:] = p['rho']*x_sim[(i-1),:] + p['phi_e']*np.sqrt(sig2_sim[(i-1),:])*e_sim[i,:]    
        delc_sim[i,:] = p['mu_c'] + x_sim[(i-1),:] + np.sqrt(sig2_sim[(i-1),:])*eta_sim[i,:]
        deld_sim[i,:] = p['mu_d'] + p['phi']*x_sim[(i-1),:] + \
                               p['pi']*np.sqrt(sig2_sim[(i-1),:])*eta_sim[i,:] + \
                               p['xi']*np.sqrt(sig2_sim[(i-1),:])*ud_sim[i,:]

    # price-cons ratio
    z_sim = p['A0'] + p['A1']*x_sim + p['A2']*sig2_sim
    zm_sim = p['A0_m'] + p['A1_m']*x_sim + p['A2_m']*sig2_sim
    
    
    # returns
    rc_sim = p['kappa_0'] + p['kappa_1']*z_sim[1:,] + delc_sim[1:,] - z_sim[0:-1,]
    rd_sim = p['kappa_0_m'] + p['kappa_1_m']*zm_sim[1:,] + deld_sim[1:,] - zm_sim[0:-1,]

    rf_sim = np.zeros((p['Tsim'], p['Nsim']))
    var_sdf_sim = (((p['lambda_m_eta'])**2 + (p['lambda_m_e'])**2)*sig2_sim \
                                           + ((p['lambda_m_w'])**2)*p['sig_w']**2)
    
    E_z_sim = p['kappa_0'] + p['kappa_1']*(p['A0']+p['A1']*p['rho']*x_sim + p['A2']*((p['sig_bar']**2) \
                           + p['nu']*(sig2_sim - (p['sig_bar']**2)))) \
                           + p['mu_c'] + x_sim - z_sim

    rf_sim =  - p['theta']*np.log(p['delta']) \
              + (p['theta']/p['psi'])*(p['mu_c'] + x_sim) \
              + (1-p['theta'])*(E_z_sim) \
              - 0.5*var_sdf_sim    
            
    return {'eta_sim':eta_sim, 
            'e_sim':e_sim, 
            'w_sim':w_sim, 
            'ud_sim':ud_sim, 
            'sig2_sim':sig2_sim,
            'x_sim':x_sim,
            'delc_sim':delc_sim,
            'deld_sim':deld_sim,
            'z_sim':z_sim,
            'zm_sim':zm_sim,
            'rc_sim':rc_sim,
            'rd_sim':rd_sim,
            'rf_sim':rf_sim,            
           }
